In [18]:
from keras.applications import inception_v3
from keras import backend as K
import tensorflow as tf
import numpy as np
from PIL import Image
import scipy
from keras.preprocessing import image

K.set_learning_phase(0)
tf.compat.v1.disable_eager_execution() #eager exec. muss disabled werden sonst geht K.set_learning_phase nicht mehr

model=inception_v3.InceptionV3(weights='imagenet',include_top=False)
layer_contributions={'mixed2':0.2,'mixed3':3.,'mixed4':2.,'mixed5':1.5,}
layer_dict=dict([(layer.name,layer) for layer in model.layers]) #fügt jedem layernamen die zugehörige instanz hinzu in einem dict
loss=K.variable(0.) #verlustvariable, beiträge der layer werden hinzuaddiert
for layer_name in layer_contributions:
  coeff=layer_contributions[layer_name]
  activation=layer_dict[layer_name].output
  scaling=K.prod(K.cast(K.shape(activation),'float32'))
  loss = loss+coeff*K.sum(K.square(activation[:,2:-2,2:-2,:]))/scaling
dream = model.input
grads=K.gradients(loss,dream)[0]
grads=grads/K.maximum(K.mean(K.abs(grads)),1e-7)
outputs=[loss,grads]
fetch_loss_and_grads=K.function([dream],outputs)
def eval_loss_and_grads(x):
    outs=fetch_loss_and_grads([x])
    loss_value=outs[0]
    grads_values=outs[1]
    return loss_value, grads_values
def gradient_ascent(x,iterations,step,max_loss=None):
    for i in range(iterations):
        loss_value,grad_values=eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('Der Wert der Verlustfunktion ist bei ',i,':',loss_value)
        x = x+step*grad_values
    return x

def resize_img(img,size):
  img=np.copy(img)
  factors=(1,float(size[0])/img.shape[1],float(size[1])/img.shape[2],1)
  return scipy.ndimage.zoom(img,factors,order=1)
def save_img(img,fname):
  pil_img=deprocess_image(np.copy(img))
  #scipy.misc.imsave(fname,pil_img)
  img = Image.fromarray(pil_img)
  img.save("output_image.jpg")
def preprocess_image(image_path):
  img=image.load_img(image_path)
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  img=inception_v3.preprocess_input(img)
  return img
def deprocess_image(x):
  if K.image_data_format()=='channels_first':
    x=x.reshape((3,x.shape[2],x.shape[3]))
    x=x.transpose((1,2,0))
  else:
    x=x.reshape((x.shape[1],x.shape[2],3))
  x=x/2
  x=x+0.5
  x=x*255.
  x=np.clip(x,0,255).astype('uint8')
  return x

step = 0.01
num_octave=3
octave_scale=1.4
iterations=20
max_loss=10.

base_image_path='/Users/oliverzimmermann/Desktop/VisKomm3_A5/Berthold.Hannes.VK3.WiSe23.A5/Nightmare.jpg'
img=preprocess_image(base_image_path)
original_shape=img.shape[1:3]
successive_shapes=[original_shape]
for i in range(1,num_octave):
    shape=tuple([int(dim/(octave_scale**i))
        for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes=successive_shapes[::-1]
original_img=np.copy(img)
shrunk_original_img=resize_img(img, successive_shapes[0])
for shapes in successive_shapes:
    print('Shape verarbeiten',shape)
    img=resize_img(img,shape)
    img=gradient_ascent(img,iterations=iterations,step=step,max_loss=max_loss)
    upscaled_shrunk_original_img=resize_img(shrunk_original_img,shape)
    same_size_original=resize_img(original_img,shape)
    lost_detail= same_size_original-upscaled_shrunk_original_img
    img = img + lost_detail
    shrunk_original_img=resize_img(original_img,shape)
    save_img(img,fname='Dream'+str(shape)+'.png')
save_img(img,fname='finalImage.png')


2024-06-19 00:24:03.777869: W tensorflow/c/c_api.cc:304] Operation '{name:'batch_normalization_1111/beta/Assign' id:57392 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_1111/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](batch_normalization_1111/beta, batch_normalization_1111/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Shape verarbeiten (988, 1318)


2024-06-19 00:24:18.053929: W tensorflow/c/c_api.cc:304] Operation '{name:'Variable_9/Assign' id:59524 op device:{requested: '', assigned: ''} def:{{{node Variable_9/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](Variable_9, Variable_9/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Der Wert der Verlustfunktion ist bei  0 : 1.5367161
Der Wert der Verlustfunktion ist bei  1 : 2.0252647
Der Wert der Verlustfunktion ist bei  2 : 2.748859
Der Wert der Verlustfunktion ist bei  3 : 3.6859145
Der Wert der Verlustfunktion ist bei  4 : 4.697443
Der Wert der Verlustfunktion ist bei  5 : 5.7354937
Der Wert der Verlustfunktion ist bei  6 : 6.7669625
Der Wert der Verlustfunktion ist bei  7 : 7.8102484
Der Wert der Verlustfunktion ist bei  8 : 8.807317
Der Wert der Verlustfunktion ist bei  9 : 9.79586
Shape verarbeiten (988, 1318)
Shape verarbeiten (988, 1318)
